In [1]:
import torch
import pandas as pd
from pandas import DataFrame
import torch.nn as nn
from skimage import io, transform, color
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import dataset, dataloader, Dataset
from torchvision import transforms, utils

In [2]:
torch.cuda.is_available()

True

In [ ]:
# https://github.com/Spandan-Madan/Pytorch_fine_tuning_Tutorial/blob/master/main_fine_tuning.py
# looks how the model is trained

In [ ]:
!wget http://saliency.mit.edu/testSet.zip -O /content/drive/MyDrive/data/testSet.zip

In [ ]:
!unzip /content/drive/MyDrive/data/testSet.zip -d /content/drive/MyDrive/data/q1

In [ ]:
!git clone https://github.com/1-1is0/deep-projects

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
from dataloader import load_data, ImageDataset, Rescale, ToTensor

# path = "/content/drive/MyDrive/data/q1/testSet/Stimuli/*/*.jpg"
path = "data/Stimuli/*/*.jpg"
train_df, test_df = load_data(path)
print("Train:", train_df.shape)
print("Test:", test_df.shape)

Train: (85, 3)
Test: (15, 3)


In [15]:
train_dataset = ImageDataset(
    train_df,
    transform=transforms.Compose(
        [
            Rescale((320, 240)),
            # RandomCrop(224),
            ToTensor(),
        ]
    ),
)


test_dataset = ImageDataset(
    test_df,
    transform=transforms.Compose(
        [
            Rescale((320, 240)),
            # RandomCrop(224),
            ToTensor(),
        ]
    ),
)


trainloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=4, shuffle=True, num_workers=2
)

testloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=4, shuffle=True, num_workers=2
)

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from model import Net

net = Net().to(device)
net.initialize_weights()
print(net)

Net(
  (conv1): Conv2d(3, 48, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (local_response_norm): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (max_pool1): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(48, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (max_pool2): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv6): Conv2d(256, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv7): Conv2d(128, 64, kernel_size=(11, 11), stride=(1, 1), padding=(5, 5))
  (conv8): Conv2d(64, 16, kernel_size=(11, 11), stride=(1, 1), padding=(5, 5))
  (conv9): Conv2d(16, 1, kernel_size=(13, 13), stride=(1, 1), padding=(6, 6))
  (deconv): ConvTran

In [18]:
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR, ExponentialLR, StepLR
import torch.nn as nn

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# scheduler = ExponentialLR(optimizer, gamma=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
from model import train_model

dataloader = {
    "train": trainloader,
    "val": testloader,
}
dataset_size = {
    "train": len(train_dataset),
    "val": len(test_dataset),
}

res = train_model(
    data_loader=dataloader,
    model=net,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    epochs=20,
    dataset_size=dataset_size,
    path="./state"
)


print(res)

train 22
epoch 4 0.0510649425400929
running loss 1.1234287358820438
val 4
epoch 4 0.04862430295906961
running loss 0.19449721183627844
train 22
epoch 5 0.05106494228609584
running loss 1.1234287302941084
val 4
epoch 5 0.048624304588884115
running loss 0.19449721835553646
train 22
epoch 6 0.05106494190510024
running loss 1.1234287219122052
val 4
epoch 6 0.04862430552020669
running loss 0.19449722208082676
train 22
epoch 7 0.05106494143943895
running loss 1.123428711667657
val 4
epoch 7 0.04862430435605347
running loss 0.1944972174242139
train 22
epoch 8 0.05106494160877033
running loss 1.1234287153929472
val 4
epoch 8 0.048624305287376046
running loss 0.19449722114950418
train 22
epoch 9 0.051064941100776196
running loss 1.1234287042170763
val 4
epoch 9 0.04862430365756154
running loss 0.19449721463024616


In [ ]:
mm = Net()
mm.load_state_dict(torch.load("./model_01.pth"))
mm.eval()

In [ ]:
i = mm(data["image"].float())
i.shape

In [ ]:
sam = i[2, :, :, :]

In [ ]:
plt.imshow(sam.permute(1, 2, 0).detach().numpy())

In [ ]:
plt.imshow(data["fix_map"][2].permute(1, 2, 0).detach().numpy())

In [ ]:
i[0, :, :, :]

In [32]:
conv1 = nn.Conv2d(3, 96, 7, stride=1, padding=3)
local_response_norm = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75)
max_pool1 = nn.MaxPool2d((3, 3), stride=2)
conv2 = nn.Conv2d(96, 256, 5, stride=1, padding=2)
max_pool2 = nn.MaxPool2d((3, 3), stride=2, padding=0)
conv3 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
conv4 = nn.Conv2d(512, 512, kernel_size=5, stride=1, padding=2)
conv5 = nn.Conv2d(512, 512, kernel_size=5, stride=1, padding=2)
conv6 = nn.Conv2d(512, 256, kernel_size=7, stride=1, padding=3)
conv7 = nn.Conv2d(256, 128, kernel_size=11, stride=1, padding=5)
conv8 = nn.Conv2d(128, 32, kernel_size=11, stride=1, padding=5)
conv9 = nn.Conv2d(32, 1, kernel_size=13, stride=1, padding=6)
deconv = nn.ConvTranspose2d(1, 1, kernel_size=8, stride=4, bias=False)

In [33]:
img = train_dataset[0]["image"].unsqueeze(0)

In [34]:
out1 = conv1(img)
print("out1:", out1.shape)
out2 = local_response_norm(out1)
print("out2:", out2.shape)
out3 = max_pool1(out2)
print("out3:", out3.shape)
out4 = conv2(out3)
print("out4:", out4.shape)
out5 = max_pool2(out4)
print("out5:", out5.shape)
out6 = conv3(out5)
print("out6:", out6.shape)
out7 = conv4(out6)
print("out7:", out7.shape)
out8 = conv5(out7)
print("out8:", out8.shape)
out9 = conv6(out8)
print("out9:", out9.shape)
out10 = conv7(out9)
print("out10:", out10.shape)
out11 = conv8(out10)
print("out11:", out11.shape)
out12 = conv9(out11)
print("out12:", out12.shape)
out13 = deconv(out12)
print("out13:", out13.shape)

out1: torch.Size([1, 96, 320, 240])
out2: torch.Size([1, 96, 320, 240])
out3: torch.Size([1, 96, 159, 119])
out4: torch.Size([1, 256, 159, 119])
out5: torch.Size([1, 256, 79, 59])
out6: torch.Size([1, 512, 79, 59])
out7: torch.Size([1, 512, 79, 59])
out8: torch.Size([1, 512, 79, 59])
out9: torch.Size([1, 256, 79, 59])
out10: torch.Size([1, 128, 79, 59])
out11: torch.Size([1, 32, 79, 59])
out12: torch.Size([1, 1, 79, 59])
out13: torch.Size([1, 1, 320, 240])
